<a href="https://colab.research.google.com/github/jobayed80/Machine_Learning_Learning_Course/blob/main/DS_424_Lab_Assignments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Build an image classification model and predict MNIST Digits using Wide and Deep Neural Network. Your should use callback functions to implement early stopping and save the best model into appropriate format. Report the training and test accuracy and other evaluation metrics. **

---



In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models, callbacks, initializers , regularizers, optimizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, ImageDataGenerator

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense


In [ ]:
# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255


In [ ]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
# Function to build the Wide and Deep Neural Network model with specified activations and initializers
def build_wide_and_deep_model(activation='relu', initializer='glorot_uniform'):
    input_layer = layers.Input(shape=(28, 28, 1))

    # Wide Path
    flat = layers.Flatten()(input_layer)
    wide = layers.Dense(64, activation=activation, kernel_initializer=initializer)(flat)

    # Deep Path
    conv1 = layers.Conv2D(32, (3, 3), activation=activation, kernel_initializer=initializer)(input_layer)
    pool1 = layers.MaxPooling2D((2, 2))(conv1)
    conv2 = layers.Conv2D(64, (3, 3), activation=activation, kernel_initializer=initializer)(pool1)
    pool2 = layers.MaxPooling2D((2, 2))(conv2)
    flat = layers.Flatten()(pool2)
    deep = layers.Dense(64, activation=activation, kernel_initializer=initializer)(flat)

    # Merge Wide and Deep paths
    merge = layers.concatenate([wide, deep])
    output = layers.Dense(10, activation='softmax')(merge)

    model = models.Model(inputs=input_layer, outputs=output)

    return model

**Use the same datasets and model as above, but this time use different activation functions and corresponding weight initialization methods to see if there is any difference from the perspective of epochs, errors, accuracy, etc. **

---



In [ ]:
# Activation functions to try
activations = ['relu', 'sigmoid', 'tanh']

# Weight initialization methods to try
initializers = ['glorot_uniform', 'he_normal', 'lecun_normal']


In [ ]:
model = build_wide_and_deep_model()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Define callback functions
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)


In [ ]:

# Train the model with callback functions
history = model.fit(train_images, train_labels, epochs=2, batch_size=64,
                    validation_split=0.2, callbacks=[early_stopping, model_checkpoint])



In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels)

# Print test accuracy and other evaluation metrics
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
print(f'Test Loss: {test_loss:.4f}')


In [ ]:
# You can load the best model from the saved checkpoint
best_model = models.load_model('best_model.h5')

In [ ]:
# Make predictions on the test set using the best model
predictions = best_model.predict(test_images)

In [ ]:
# Display the results
for i in range(5):
    plt.imshow(test_images[i].reshape(28, 28), cmap='gray')
    plt.title(f'Predicted: {predictions[i]}, Actual: {tf.argmax(test_labels[i])}')
    plt.show()

**Use the same dataset, model, activation functions, and weight initialization methods. However, now try different optimizers such as Momentum, Nesterov, AdaGrad, RMSProp, ADAM, and so on, and check whether you can reduce the training time, or increase the accuracy.**

---



In [ ]:
# Optimizers to try
optimizers = ['sgd', 'momentum', 'nesterov', 'adagrad', 'rmsprop', 'adam']

In [ ]:
# Build a Wide and Deep Neural Network
model = models.Sequential()
model.add(layers.Flatten(input_shape=(28, 28, 1)))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
# Compile the model with different optimizers
optimizers_list = ['sgd', 'adagrad', 'rmsprop', 'adam']

In [ ]:
for optimizer_name in optimizers_list:
    model.compile(optimizer=optimizer_name,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
 # Train the model
    historys = model.fit(train_images, train_labels, epochs=2, batch_size=64, validation_data=(test_images, test_labels))

In [ ]:

    # Evaluate the model
    test_loss, test_acc = model.evaluate(test_images, test_labels)
    print(f"\nOptimizer: {optimizer_name}")
    print(f"Test Accuracy: {test_acc * 100:.2f}%")

In [ ]:
# Visualize the training process
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')

**4	Use the same dataset, model, activation functions, weight initialization methods, and optimizer as before. However, this time implement different regularization methods such as L1, L2, and Dropout to reduce overfitting. See Chapter 12 of the Textbook 1.**

---



In [ ]:
# Build a Wide and Deep Neural Network with regularization
model = models.Sequential()
model.add(layers.Flatten(input_shape=(28, 28, 1)))
model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
# Compile the model with an optimizer
optimizer_name = 'adam'
model.compile(optimizer=optimizer_name,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(train_images, train_labels, epochs=2, batch_size=64, validation_data=(test_images, test_labels))


In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"\nOptimizer: {optimizer_name}")
print(f"Test Accuracy: {test_acc * 100:.2f}%")

In [ ]:
# Visualize the training process
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')

In [ ]:
# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'], loc='upper left')

plt.tight_layout()
plt.show()

**Implement ResNet-34 architecture from the scratch using Keras Sequential API. Now train the network to predict on MNIST Fashion dataset. Evaluate your model using appropriate metrics. **

---



In [ ]:
# Define the Residual Block
def residual_block(x, filters, kernel_size=3, strides=1):
    y = layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    y = layers.BatchNormalization()(y)
    y = layers.Activation('relu')(y)

    y = layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, padding='same')(y)
    y = layers.BatchNormalization()(y)

    # Add the shortcut connection
    if strides > 1:
        x = layers.Conv2D(filters, kernel_size=1, strides=strides, padding='same')(x)
        x = layers.BatchNormalization()(x)

    # Add the shortcut connection to the main path
    y = layers.add([x, y])
    y = layers.Activation('relu')(y)
    return y

In [ ]:
# Build the ResNet-34 model
model = models.Sequential()

In [ ]:
# Initial Convolutional Layer
model.add(layers.Conv2D(64, kernel_size=7, strides=2, input_shape=(28, 28, 1), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=3, strides=2, padding='same'))

In [ ]:
# Residual Stages
stage_filters = [64, 128, 256, 512]
for filters in stage_filters:
    model.add(residual_block(model.layers[-1].output, filters, strides=2))

In [ ]:
# Global Average Pooling and Dense Layer
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(10, activation='softmax'))


In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:

# Train the model
history = model.fit(train_images, train_labels, epochs=2, batch_size=64, validation_data=(test_images, test_labels))

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

In [ ]:
# Visualize the training process
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')

6. Now train the same dataset and the same ResNet-34, but use pre-trained weights. In other words, use transfer learning to train the same model and determine whether there is any improvement in training time, accuracy, or any other aspect.

---

**bold text**

In [ ]:
# Repeat single-channel images to have three channels
train_images_rgb = tf.image.grayscale_to_rgb(tf.convert_to_tensor(train_images))
test_images_rgb = tf.image.grayscale_to_rgb(tf.convert_to_tensor(test_images))


In [ ]:
# Resize images to meet the minimum size required by ResNet50
train_images_rgb = tf.image.resize(train_images_rgb, (32, 32))
test_images_rgb = tf.image.resize(test_images_rgb, (32, 32))


In [ ]:
# Build the base ResNet-50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


In [ ]:
# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
# Build the transfer learning model
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(10, activation='softmax'))


In [ ]:
# Compile the model
model.compile(optimizer=optimizers.Adam(lr=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model with data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.1
)


In [ ]:
datagen.fit(train_images_rgb)

In [ ]:
model.fit(datagen.flow(train_images_rgb, train_labels, batch_size=32),
          epochs=1,
          validation_data=(test_images_rgb, test_labels))

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images_rgb, test_labels)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

In [ ]:
# Visualize the training process
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')

In [ ]:
# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'], loc='upper left')

plt.tight_layout()
plt.show()

**7. Generate an artificial time series data and predict it using simple, deep Recurrent Neural Network, or LSTM cells.**

---



In [ ]:
# Function to generate time series data
def generate_time_series(signal_length=1000):
    time = np.arange(0, signal_length)
    signal = np.sin(0.1 * time) + 0.2 * np.random.randn(signal_length)
    return signal

In [ ]:
# Function to create sequences for time series prediction
def create_sequences(data, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        seq = data[i : i + sequence_length]
        target = data[i + sequence_length]
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

In [ ]:
# Generate time series data
signal_length = 1000
signal = generate_time_series(signal_length)


In [ ]:
# Set sequence length for prediction
sequence_length = 20


In [ ]:
# Create sequences for training
sequences, targets = create_sequences(signal, sequence_length)


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sequences, targets, test_size=0.2, random_state=42)

In [ ]:
# Reshape the data for RNN and LSTM models
X_train = X_train.reshape(-1, sequence_length, 1)
X_test = X_test.reshape(-1, sequence_length, 1)


In [ ]:
# Build Simple RNN model
model_rnn = Sequential([
    SimpleRNN(50, input_shape=(sequence_length, 1)),
    Dense(1)
])

In [ ]:
model_rnn.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
# Train Simple RNN model
history_rnn = model_rnn.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), verbose=2)


In [ ]:
# Build LSTM model
model_lstm = Sequential([
    LSTM(50, input_shape=(sequence_length, 1)),
    Dense(1)
])

In [ ]:
model_lstm.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
# Train LSTM model
history_lstm = model_lstm.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), verbose=2)

In [ ]:
# Visualize the training loss
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history_rnn.history['loss'], label='RNN Training Loss')
plt.plot(history_rnn.history['val_loss'], label='RNN Validation Loss')
plt.title('Simple RNN Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
# Predict using both models
X_test_reshaped = X_test.reshape((-1, sequence_length, 1))

# Predictions using Simple RNN
predictions_rnn = model_rnn.predict(X_test_reshaped)

# Predictions using LSTM
predictions_lstm = model_lstm.predict(X_test_reshaped)


In [ ]:
# Plot the predictions
plt.figure(figsize=(12, 6))
plt.plot(y_test, label='True Values', linestyle='dashed', color='black')
plt.plot(predictions_rnn, label='RNN Predictions', linestyle='dashed', color='blue')
plt.plot(predictions_lstm, label='LSTM Predictions', linestyle='dashed', color='red')
plt.title('Time Series Prediction with RNN and LSTM')
plt.xlabel('Time Steps')
plt.ylabel('Values')
plt.legend()
plt.show()

**8. Generate Shakespearean texts using Character Recurrent Neural Network. Develop the training dataset by splitting and chopping the training texts. Then develop a Char-RNN model to generate fake Shakespearean texts.**

---



In [ ]:
import tensorflow as tf
import numpy as np
import requests
import string

In [ ]:
# Download Shakespearean text
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
shakespeare_text = requests.get(url).text

In [ ]:
# Preprocess the text
shakespeare_text = shakespeare_text.lower()
shakespeare_text = shakespeare_text.translate(str.maketrans("", "", string.punctuation))

In [ ]:
# Create character-to-index and index-to-character mappings
chars = sorted(list(set(shakespeare_text)))
char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for idx, char in enumerate(chars)}

In [ ]:
# Create sequences for training
seq_length = 100
step = 1
sequences = []
next_chars = []

In [ ]:
for i in range(0, len(shakespeare_text) - seq_length, step):
    seq = shakespeare_text[i : i + seq_length]
    next_char = shakespeare_text[i + seq_length]
    sequences.append(seq)
    next_chars.append(next_char)

In [ ]:
# Convert sequences to input and target data
X = np.zeros((len(sequences), seq_length, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, seq in enumerate(sequences):
    for t, char in enumerate(seq):
        X[i, t, char_to_idx[char]] = 1
    y[i, char_to_idx[next_chars[i]]] = 1

In [ ]:
# Build the Char-RNN model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(seq_length, len(chars))),
    tf.keras.layers.Dense(len(chars), activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy")

In [ ]:
# Generate text using the trained model
def generate_text(seed_text, length=500, temperature=1.0):
    generated_text = seed_text

    for _ in range(length):
        x_pred = np.zeros((1, seq_length, len(chars)))

        for t, char in enumerate(seed_text):
            x_pred[0, t, char_to_idx[char]] = 1.0

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = idx_to_char[next_index]

        generated_text += next_char
        seed_text = seed_text[1:] + next_char

    return generated_text

In [ ]:
# Helper function to sample from the predicted probabilities
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
# Generate Shakespearean text
seed_text = "shall i compare thee to a summers day\n"
generated_text = generate_text(seed_text, length=1000, temperature=0.5)

In [ ]:
# Print and save the generated text
print(generated_text)

# Save the model for future use
model.save("char_rnn_shakespeare_model.h5")

# 9. Download the IMBD dataset and perform a sentiment analysis on the movie reviews. You can use pre-trained embeddings if you want.

---

**bold text**

In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import get_file


In [ ]:
# Download and load the IMDB dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)


In [ ]:
# Download the GloVe word embeddings
# Specify the correct path to the extracted glove file
glove_txt_path = '/path/to/glove.6B.100d.txt'  # Update with the actual path to your glove file

embedding_index = {}
with open(glove_txt_path) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embedding_index[word] = coefs


In [ ]:
# Create an embedding matrix
word_index = imdb.get_word_index()
embedding_dim = 100
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
# Pad sequences to a fixed length
max_len = 200
x_train_padded = pad_sequences(x_train, maxlen=max_len)
x_test_padded = pad_sequences(x_test, maxlen=max_len)


In [ ]:
# Build the model
model = Sequential()
model.add(Embedding(len(word_index) + 1, embedding_dim, input_length=max_len, weights=[embedding_matrix], trainable=False))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [ ]:
# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [ ]:
# Train the model
history = model.fit(x_train_padded, y_train, epochs=2, batch_size=32, validation_split=0.2)


In [ ]:

# Evaluate the model on test data
loss, accuracy = model.evaluate(x_test_padded, y_test)
print(f"Test Accuracy: {accuracy}")

In [ ]:
# Visualize training process
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

**10. Suppose you are going to classify images of at cats and dogs. Now collect images of different cats and dogs from online or real life. You and your classmates may develop this dataset together. Then organize the dataset in ImageNet folder structure.**

---



In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras import layers
import cv2, os
from tqdm import tqdm
from random import shuffle
import shutil

In [ ]:
!wget "https://cainvas-static.s3.amazonaws.com/media/user_data/tanmay/dogvcat.zip"
!unzip -qo dogvcat.zip

In [ ]:
train_dir= "./datset/training"
test_dir= "./datset/testing/"
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split= 0.2)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(50,50), color_mode='grayscale', batch_size=20, class_mode='binary', subset= 'training')
val_generator = train_datagen.flow_from_directory(train_dir, target_size=(50,50), color_mode='grayscale', batch_size=20, class_mode='binary', subset= 'validation')

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, kernel_size=(3,3), activation='relu',input_shape=(50,50,1), padding='same'))
# model.add(LeakyReLU(alpha=0.1))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', padding='same'))
# model.add(tf.keras.activations.relu(alpha=0.1))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
# model.add(tf.keras.activations.relu(alpha=0.1))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
# model.add(tf.keras.activations.relu(alpha=0.1))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
# model.add(tf.keras.activations.relu(alpha=0.1))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
# Model Compilation
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train_generator, epochs=2, validation_data=val_generator, validation_steps=50)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

**11	Use the TensorFlow Data API to load your dataset that you developed in the previous section. Using appropriate preprocessing and data augmentation techniques as necessary.**

---



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers

In [ ]:
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

In [ ]:
num_classes = metadata.features['label'].num_classes
print(num_classes)

In [ ]:
get_label_name = metadata.features['label'].int2str

image, label = next(iter(train_ds))
_ = plt.imshow(image)
_ = plt.title(get_label_name(label))

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

In [ ]:
# Add the image to a batch.
image = tf.cast(tf.expand_dims(image, 0), tf.float32)

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = data_augmentation(image)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_image[0])
  plt.axis("off")

In [ ]:
def random_invert_img(x, p=0.5):
  if  tf.random.uniform([]) < p:
    x = (255-x)
  else:
    x
  return x

In [ ]:
def random_invert(factor=0.5):
  return layers.Lambda(lambda x: random_invert_img(x, factor))

random_invert = random_invert()

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = random_invert(image)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_image[0].numpy().astype("uint8"))
  plt.axis("on")

In [ ]:
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

In [ ]:
image, label = next(iter(train_ds))
_ = plt.imshow(image)
_ = plt.title(get_label_name(label))

In [ ]:
grayscaled = tf.image.rgb_to_grayscale(image)
visualize(image, tf.squeeze(grayscaled))
_ = plt.colorbar()

**12. Develop at least 3 different CNN architectures such as VGG, ResNet, MobileNet, etc., using Keras and train your model on the dataset. Report the appropriate evaluation metrics of your models.**

---



In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras import layers
import cv2, os
from tqdm import tqdm
from random import shuffle
import shutil

In [ ]:
!wget "https://cainvas-static.s3.amazonaws.com/media/user_data/tanmay/dogvcat.zip"
!unzip -qo dogvcat.zip

In [ ]:
train_dir= "./datset/training"
test_dir= "./datset/testing/"
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split= 0.2)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(50,50), color_mode='grayscale', batch_size=20, class_mode='binary', subset= 'training')
val_generator = train_datagen.flow_from_directory(train_dir, target_size=(50,50), color_mode='grayscale', batch_size=20, class_mode='binary', subset= 'validation')

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, kernel_size=(3,3), activation='relu',input_shape=(50,50,1), padding='same'))
# model.add(LeakyReLU(alpha=0.1))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', padding='same'))
# model.add(tf.keras.activations.relu(alpha=0.1))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
# model.add(tf.keras.activations.relu(alpha=0.1))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
# model.add(tf.keras.activations.relu(alpha=0.1))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
# model.add(tf.keras.activations.relu(alpha=0.1))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
# Model Compilation
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train_generator, epochs=5, validation_data=val_generator, validation_steps=50)


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()